In [1]:
pip install scikit-multilearn

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for scikit-multilearn from https://files.pythonhosted.org/packages/bb/1f/e6ff649c72a1cdf2c7a1d31eb21705110ce1c5d3e7e26b2cc300e1637272/scikit_multilearn-0.2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/89.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/89.4 kB ? eta -:--:--
   ------------- -------------------------- 30.7/89.4 kB ? eta -:--:--
   ------------- -------------------------- 30.7/89.4 kB ? eta -:--:--
   ------------- -------------------------- 30.7/89.4 kB ? eta -:--:--
   ------------- -------------------------- 30.7/89.4 kB ? eta -:--:--
   ------------- -------------------------- 30.7/89.4 kB ? eta -:--:--
   ------------- -------------------------- 30.7/89.4 kB ? eta -:--:--
   ------------- -------------------------- 30.7/89.4 kB ? eta -:--:--
   --------------------------- ------------ 61.4/89.4 kB 172.4 kB/

In [5]:
pip install liac-arff


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11727 sha256=1390355493afe25d35b9eb4efd0397e9c6c5a793ab3c6d9d22bf386733209ea8
  Stored in directory: c:\users\rafae\appdata\local\pip\cache\wheels\00\23\31\5e562fce1f95aabe57f2a7320d07433ba1cd152bcde2f6a002
Successfully built liac-arff
Note: you may need to restart the kernel to use updated packages.


In [6]:

from skmultilearn.dataset import load_dataset

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
import pandas as pd

C:\Users\rafae\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [9]:
import numpy as np

In [8]:
prev_train_df = pd.read_csv("train.csv")

In [29]:
comments_df = pd.read_csv("comments_to_score.csv")
valid_df = pd.read_csv("validation_data.csv")

In [10]:
for col in ["toxic", "severe_toxic","obscene","threat","insult","identity_hate"]:
    print(col, prev_train_df[col].sum())

toxic 15294
severe_toxic 1595
obscene 8449
threat 478
insult 7877
identity_hate 1405


In [11]:
prev_train_df["severe_toxic"] = prev_train_df["severe_toxic"] * 2.5
prev_train_df["threat"] = prev_train_df["threat"] * 3
prev_train_df["identity_hate"] = prev_train_df["identity_hate"] * 2
prev_train_df["threat"] = prev_train_df["threat"] * 3

prev_train_df["total_toxic"] = prev_train_df[["toxic", "severe_toxic","obscene","threat","insult","identity_hate"]].sum(axis=1).astype(np.int64)


In [14]:
prev_train_df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,total_toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0.0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0.0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0.0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0.0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0.0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0.0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0.0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0.0,0,0,0,0,0


In [16]:
train_df = prev_train_df[["comment_text", "total_toxic"]]

In [20]:
tfvec = TfidfVectorizer(analyzer = 'char_wb', ngram_range = (3,5))
tfv = tfvec.fit_transform(train_df["comment_text"])

In [27]:
X=tfv[1:15001]
Y=train_df.loc[1:15000,['total_toxic']]

In [28]:
reg = LinearRegression().fit(X,Y)

In [30]:
tfv_comments = tfvec.transform(comments_df["text"])
pred = reg.predict(tfv_comments)

In [31]:
a=pred[:1000]

In [32]:
reg.score(X,Y) 

0.998367640805852

In [33]:
sub = pd.DataFrame()
sub["comment_id"] = comments_df["comment_id"]
sub["score"] = pred
sub.to_csv('submission.csv',index=False)
sub

,comment_id,score
0,114890,-0.000004
1,732895,1.095844
2,1139051,-1.243364
3,1434512,0.430446
4,2084821,0.420494
...,...,...
7532,504235362,0.098709
7533,504235566,-0.475848
7534,504308177,-0.228363
7535,504570375,1.355358
